In [1]:
import psycopg2 as pc
import numpy
import pandas as pd
import plotly.express as px

In [2]:
def read_query(connection, query):
    cursor = connection.cursor()
    result = None
    try:
        cursor.execute(query)
        result = cursor.fetchall()
        return result
    except Error as err:
        print(f"Error: {err}")

In [3]:
DB_HOST = "postgres_db"
DB_NAME = "test_db"
DB_USER = "root"
DB_PASS = "root"

In [ ]:
conn = pc.connect(dbname=DB_NAME, user=DB_USER, password=DB_PASS, host=DB_HOST)
cur = conn.cursor()

## `Cargando datos a la DB`

cur.execute("CREATE TABLE INVEMAR (id serial PRIMARY KEY, Fecha integer,Hora integer,Variable varchar,Valor float,Unidad varchar,Lugar varchar);")
conn.commit()
df = pd.read_csv('Data_INVEMAR.csv')
df
cols = ",".join([str(i) for i in df.columns.tolist()])

for i,row in df.iterrows():
    sql = "INSERT INTO INVEMAR (" +cols + ") VALUES (" + "%s,"*(len(row)-1) + "%s)"
    cur.execute(sql,tuple(row))
conn.commit()

## `Visualizaciones (Queries)`

In [ ]:
r=read_query(conn, "select Hora, Valor, Lugar from INVEMAR where Valor<150")
a = []
b = []
c = []
for data in r:
    a.append(data[0])
    b.append(data[1])
    c.append(data[2])
dd = {'Hora':a,'Velocidad':b, "Lugar":c }
fig = px.scatter(dd, x="Hora", y="Velocidad", color="Lugar", marginal_y="violin",
           marginal_x="box", trendline="ols", template="simple_white")#px.scatter(dd, x="Hora", y="Velocidad", color="Lugar")
fig.show()

In [ ]:
o=read_query(conn, "select Hora, avg(Valor) from INVEMAR group by Hora order by Hora")
a = []
b = []
for data in o:
    a.append(data[0])
    b.append(data[1])
dd = {'Hora':a,'Velocidad':b }
fig = px.line(dd, x="Hora", y="Velocidad", title='Promedio de velocidad por hora')
fig.show()

In [ ]:
q=read_query(conn, "select Hora, max(Valor) from INVEMAR where Lugar='Punta Betin' group by Hora order by Hora")
a = []
b = []
for data in q:
    a.append(data[0])
    b.append(data[1])
dd = {'Hora':a,'Velocidad':b }
fig = px.line(dd, x="Hora", y="Velocidad", title='Maxima de velocidad por hora en Barranquilla')
fig.show()

In [ ]:
cur.close()
conn.close()